# Videos Downloader

In [1]:
import os
import pandas as pd
import yt_dlp
from googleapiclient.discovery import build
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

# Set up the API client
api_key = os.getenv('API_KEY')
youtube = build('youtube', 'v3', developerKey=api_key)
channel_id = 'UCMwXnEIdNOe_nEcfqzjOO7A'


### Get Channel Videos

In [2]:
def get_uploads_playlist_id(channel_id):
    """Retrieve the uploads playlist ID for a given channel ID."""
    try:
        channel_response = youtube.channels().list(
            part='contentDetails',
            id=channel_id
        ).execute()
        uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        return uploads_playlist_id
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

uploads_playlist = get_uploads_playlist_id(channel_id)
print(f"Uploads playlist ID: {uploads_playlist}")


Uploads playlist ID: UUMwXnEIdNOe_nEcfqzjOO7A


In [3]:

def get_videos_from_playlist(playlist_id):
    """Retrieve all videos from a given playlist ID."""
    videos = []
    next_page_token = None

    while True:
        try:
            playlist_items_response = youtube.playlistItems().list(
                part='snippet',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            ).execute()

            videos += playlist_items_response['items']
            next_page_token = playlist_items_response.get('nextPageToken')

            if next_page_token is None:
                break
        except Exception as e:
            print(f"An error occurred: {e}")
            break

    return videos
videos = get_videos_from_playlist(uploads_playlist)
print(f"Total videos: {len(videos)}")

Total videos: 78


# Save Information into a dataframe

In [30]:
df = pd.DataFrame([{
    'video_id': video['snippet']['resourceId']['videoId'],
    'channel_title': video['snippet']['channelTitle'],
    'title': video['snippet']['title'],
    'description': video['snippet']['description'],
    'thumbnail_url': video['snippet']['thumbnails']['maxres']['url'] if 'maxres' in video['snippet']['thumbnails'] else video['snippet']['thumbnails']['high']['url'],
} for video in videos])
df


,video_id,channel_title,title,description,thumbnail_url
0,Ad44riH5CgI,استوديو القمة Alqimmah Studio,منال العلا بالمرهفات القواضب | إلقاء: أسامة ال...,منال العلا بالمرهفات القواضب\nقصيدة لعلي بن ال...,https://i.ytimg.com/vi/Ad44riH5CgI/maxresdefau...
1,38kjm-kMjy0,استوديو القمة Alqimmah Studio,سورة الأنفال I القارئ: محمد السعيد المتولي,تسجيل استديو القمة لسورة الأنفال\n بصوت القار...,https://i.ytimg.com/vi/38kjm-kMjy0/maxresdefau...
2,5hdN4hWWw4A,استوديو القمة Alqimmah Studio,بانت سعاد- كعب بن زهير | إلقاء: أسامة الواعظ,بانت سعاد فقلبي اليوم متبول\nقصيدة لكعب بن زهي...,https://i.ytimg.com/vi/5hdN4hWWw4A/maxresdefau...
3,q5NJpVFDbm4,استوديو القمة Alqimmah Studio,أشبت فتىً وقبل ضحى المشيب I كلمات الشاعر: إيها...,أشبت فتىً وقبل ضحى المشيب\nكلمات الشاعر: إيهاب...,https://i.ytimg.com/vi/q5NJpVFDbm4/maxresdefau...
4,ZC3EMxSj8zE,استوديو القمة Alqimmah Studio,سورة الأحقاف I القارئ: أسامة شبانة,تسجيل استديو القمة لسورة الأحقاف\n بصوت القارئ...,https://i.ytimg.com/vi/ZC3EMxSj8zE/maxresdefau...
...,...,...,...,...,...
73,Cn8P7mYLzj0,استوديو القمة Alqimmah Studio,لا يحمل الحقد من تعلو به الرتب- أداء: خالد الشرفي,أبيات تنسب للشاعر الجاهلي عنترة بن شداد، فيها ...,https://i.ytimg.com/vi/Cn8P7mYLzj0/maxresdefau...
74,QdU9vAJp7ww,استوديو القمة Alqimmah Studio,سورة الحجرات- للقارئ عبدالرحمن عادل فهمي,,https://i.ytimg.com/vi/QdU9vAJp7ww/maxresdefau...
75,-xsmrhQS5Io,استوديو القمة Alqimmah Studio,لكل شيء إذا ما تم نقصان- قصيدة في رثاء الأندلس...,كلمات القصيدة لأبي البقاء الرندي\nبصوت: أسامة ...,https://i.ytimg.com/vi/-xsmrhQS5Io/maxresdefau...
76,gHB5admHwrE,استوديو القمة Alqimmah Studio,قصيدة حسان بن ثابت في رثاء النبي صلى الله عليه...,هذه القصيدة تنسب للصحابي الجليل حسان بن ثابت -...,https://i.ytimg.com/vi/gHB5admHwrE/hqdefault.jpg


In [ ]:
# dump the data to a JSON file 
df.to_json('data.json', orient='records', force_ascii=False)

# Download Videos/ Audios

In [32]:
## find videos with title containing a specific keyword
keyword = 'أسامة الواعظ'
df_filtered = df[df['title'].str.contains(keyword, case=False)]
# replace title with keyword
df_filtered.loc[:, 'channel_title'] = keyword
df_filtered

,video_id,channel_title,title,description,thumbnail_url
0,Ad44riH5CgI,أسامة الواعظ,منال العلا بالمرهفات القواضب | إلقاء: أسامة ال...,منال العلا بالمرهفات القواضب\nقصيدة لعلي بن ال...,https://i.ytimg.com/vi/Ad44riH5CgI/maxresdefau...
2,5hdN4hWWw4A,أسامة الواعظ,بانت سعاد- كعب بن زهير | إلقاء: أسامة الواعظ,بانت سعاد فقلبي اليوم متبول\nقصيدة لكعب بن زهي...,https://i.ytimg.com/vi/5hdN4hWWw4A/maxresdefau...
12,01v6T9wghjw,أسامة الواعظ,على قدر أهل العزم تأتي العزائم | إلقاء: أسامة ...,على قدر أهل العزم تأتي العزائم\nقصيدة للمتنبي ...,https://i.ytimg.com/vi/01v6T9wghjw/maxresdefau...
17,lU12RCIusqM,أسامة الواعظ,ألا ما لعينك أم ما لها | الخنساء | إلقاء: أسام...,ألا ما لعينك أم ما لها\nقصيدة للخنساء (ت:٢٤هـ)...,https://i.ytimg.com/vi/lU12RCIusqM/maxresdefau...
24,Y0tWGG-SzCw,أسامة الواعظ,أضحى التنائي بديلاً من تدانينا - إلقاء: أسام...,أضحى التنائي بديلاً من تدانينا\nقصيدة لابن زيد...,https://i.ytimg.com/vi/Y0tWGG-SzCw/maxresdefau...
26,r6VpT0Lid0E,أسامة الواعظ,حكايات أحمد شوقي {١٠ قصائد} - إلقاء أسامة الواعظ,الحكايات من ديوان أحمد شوقي\n\nإلقاء ومونتاج: ...,https://i.ytimg.com/vi/r6VpT0Lid0E/maxresdefau...
31,oWzo-SsqOVY,أسامة الواعظ,لامية ابن الوردي - إلقاء أسامة الواعظ,لامية ابن الوردي ت749هـ\n\nإلقاء ومونتاج: أسام...,https://i.ytimg.com/vi/oWzo-SsqOVY/maxresdefau...
32,_qXLo4wTk9E,أسامة الواعظ,نظم مثلث قطرب للبهنسي مع شرحه لابن زريق - إلقا...,نظم مثلث قطرب \nلسديد الدين البهنسي\nمع شرحه ل...,https://i.ytimg.com/vi/_qXLo4wTk9E/maxresdefau...
34,y6CKU_ob1xU,أسامة الواعظ,لعينيك ما يلقى الفؤاد وما لقي - للمتنبي I إلقا...,لعينيك ما يلقى الفؤاد وما لقي\nقصيدة للمتنبي (...,https://i.ytimg.com/vi/y6CKU_ob1xU/maxresdefau...
37,GLaWhcwEHFA,أسامة الواعظ,ألا من رأى الطفل المفارق أمه - لابن الزيات - إ...,ألا من رأى الطفل المفارق أمه\nقصيدة لابن الزيا...,https://i.ytimg.com/vi/GLaWhcwEHFA/hqdefault.jpg


In [ ]:
# download the videos
ydl_opts = {
    'format': 'best',
    'outtmpl': 'videos/%(title)s.%(ext)s',
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    for video_id in df['video_id']:
        video_url = f'https://www.youtube.com/watch?v={video_id}'
        ydl.download([video_url])
        

In [33]:
import yt_dlp
import os
import pandas as pd

# Path to ffmpeg and ffprobe
ffmpeg_path = os.getenv('FFMPEG_PATH')
ffprope_path = os.getenv('FFPROBE_PATH')

# Verify that ffmpeg and ffprobe exist at the specified locations
if not os.path.exists(ffmpeg_path):
    raise FileNotFoundError(f"ffmpeg not found at {ffmpeg_path}")
if not os.path.exists(ffprope_path):
    raise FileNotFoundError(f"ffprobe not found at {ffprope_path}")

# Adjust yt_dlp options
ydl_opts = {
    'format': 'bestaudio',
    'outtmpl': 'music/%(title)s.%(ext)s',
    'ffmpeg_path': ffmpeg_path,  # Specify ffmpeg location
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }, {
        'key': 'EmbedThumbnail',
    }, {
        'key': 'FFmpegMetadata',
        'add_metadata': True,
    }],
    'postprocessor_args': [
        '-metadata', 'artist=CHANNEL_NAME_PLACEHOLDER'
    ]
}

# Function to download videos
def download_videos_as_audio(df):
    for idx, row in df.iterrows():
        video_id = row['video_id']
        # Update artist metadata
        ydl_opts['postprocessor_args'] = ['-metadata', f'artist={row["channel_title"]}', 
                                          '-metadata', f'title={row["title"]}',
                                            ]
                                          
        
        # Use context manager to apply options
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            video_url = f'https://www.youtube.com/watch?v={video_id}'
            ydl.download([video_url])

# Call the function to download videos
download_videos_as_audio(df_filtered)

[youtube] Extracting URL: https://www.youtube.com/watch?v=Ad44riH5CgI
[youtube] Ad44riH5CgI: Downloading webpage
[youtube] Ad44riH5CgI: Downloading ios player API JSON
[youtube] Ad44riH5CgI: Downloading m3u8 information
[info] Ad44riH5CgI: Downloading 1 format(s): 251
[download] Destination: music\منال العلا بالمرهفات القواضب ｜ إلقاء： أسامة الواعظ.webm
[download] 100% of    7.88MiB in 00:00:02 at 3.42MiB/s     
[ExtractAudio] Destination: music\منال العلا بالمرهفات القواضب ｜ إلقاء： أسامة الواعظ.mp3
Deleting original file music\منال العلا بالمرهفات القواضب ｜ إلقاء： أسامة الواعظ.webm (pass -k to keep)
[EmbedThumbnail] There are no thumbnails on disk
[Metadata] Adding metadata to "music\منال العلا بالمرهفات القواضب ｜ إلقاء： أسامة الواعظ.mp3"
[youtube] Extracting URL: https://www.youtube.com/watch?v=5hdN4hWWw4A
[youtube] 5hdN4hWWw4A: Downloading webpage
[youtube] 5hdN4hWWw4A: Downloading ios player API JSON
[youtube] 5hdN4hWWw4A: Downloading m3u8 information
[info] 5hdN4hWWw4A: Downloading